<a href="https://colab.research.google.com/drive/1C9-NoEPQv3dQuby80LBC76JrEE97BRZ3#scrollTo=MhntZ9TDSf3g" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pour rouler et voir les résultats.

# Model Evaluation
Notebook to test model performance given transformed dataset and results for the four tasks. Metrics used:
- Fretchet Inception Distance (VanGogh)
- Kernel Inception Distance (VanGogh)
- Peak Signal Noise Ratio (deblurring and denoising)
- NIQE (deblocking)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from skimage.metrics import peak_signal_noise_ratio
from statistics import mean
from PIL import Image
import os, sys, shutil
import numpy as np
import skimage.transform as st
from skimage.util import img_as_ubyte
import math

!pip install tabulate
from tabulate import tabulate

!pip install scikit-video
from skvideo import measure

"Sci-kit video uses deprecated version of scipy"
!pip install scipy==1.2.3

!pip install torch-fidelity

In [ ]:
# Load files

# Paths
URBAN_ORIG_PATH = "/content/drive/MyDrive/Image_to_Image_Project/data/originals/originals.npy"
VANGOGH_ORIG_PATH = "/content/vangogh_originals"

CYCLEGAN_DEBLURRED_PATH = "cyclegan_deblurred"
CYCLEGAN_DENOISED_PATH = "cyclegan_denoised"
CYCLEGAN_DEBLOCKED_PATH = "cyclegan_deblocked"

UGATIT_DEBLURRED_PATH = "ugatit_deblurred"
UGATIT_DENOISED_PATH = "ugatit_denoised"
UGATIT_DEBLOCKED_PATH = "ugatit_deblocked"

# CycleGAN
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/CycleGAN_results_blurred.zip -d cyclegan_deblurred
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/CycleGAN_results_compressed.zip -d cyclegan_deblocked
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/CycleGAN_results_noisy.zip -d cyclegan_denoised
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/CycleGAN_results_vangogh.zip -d cyclegan_vangogh

# U-GAT-IT
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/UGATIT_results_blurred.zip -d ugatit_deblurred
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/UGATIT_results_compressed.zip -d ugatit_deblocked
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/UGATIT_results_noisy.zip -d ugatit_denoised
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/New_data/UGATIT_results_vangogh.zip -d ugatit_vangogh

# VanGogh original
!unzip -uq /content/drive/MyDrive/Image_to_Image_Project/data/vangogh_results_originals.zip -d vangogh_originals

In [ ]:
#Load original Urban100 images and resize to 256x256

size = 256
urban100_originals = np.load(URBAN_ORIG_PATH, allow_pickle=True) 
urban100_resized = []

for img in urban100_originals:
  urban100_resized.append(st.resize(img, (size, size)))

urban100_resized = np.array(urban100_resized)
print(f"Shape of resized originals: {urban100_resized.shape}")

Shape of resized originals: (100, 256, 256, 3)


# Peak Signal to Noise Ratio 

In [ ]:
def computePSNR(DEBLURRED_PATH, DENOISED_PATH, task):
    """Compute PSNR for a specified task
    
    DEBLURRED_PATH: denoised path for the task
    DENOISED_PATH: denoised path for the task
    task (string): CycleGAN or U-GAT-IT
    """
    deblurred_psnrs = []
    denoised_psnrs = []

    # Load genereated images
    deblurred_imgs = os.listdir(DEBLURRED_PATH)
    denoised_imgs = os.listdir(DENOISED_PATH)

    # Sort images
    deblurred_imgs.sort()
    denoised_imgs.sort()

    for i, img_real in enumerate(urban100_resized):
      img_real = img_as_ubyte(img_real)

      # Deblur task
      path_fake = os.path.join(DEBLURRED_PATH, deblurred_imgs[i])
      img_fake = np.array(Image.open(path_fake).convert("RGB"))
      deblurred_psnrs.append(10**(peak_signal_noise_ratio(img_real, img_fake)/10))

      # Denoise task
      path_fake = os.path.join(DENOISED_PATH, denoised_imgs[i])
      img_fake = np.array(Image.open(path_fake).convert("RGB"))
      denoised_psnrs.append(10**(peak_signal_noise_ratio(img_real, img_fake)/10))

    print(tabulate([['Deblur', 10*math.log(mean(deblurred_psnrs), 10)], 
                    ['Denoise', 10*math.log(mean(denoised_psnrs), 10)],], 
                   headers=[task, 'Mean PSNR (dB)'], tablefmt='orgtbl'))
    print("\n")

In [ ]:
#CycleGAN
computePSNR(CYCLEGAN_DEBLURRED_PATH, CYCLEGAN_DENOISED_PATH, "CycleGAN")

#U-GAT-IT
computePSNR(UGATIT_DEBLURRED_PATH, UGATIT_DENOISED_PATH, "U-GAT-IT")

| CycleGAN   |   Mean PSNR (dB) |
|------------+------------------|
| Deblur     |          23.2795 |
| Denoise    |          22.7758 |


| U-GAT-IT   |   Mean PSNR (dB) |
|------------+------------------|
| Deblur     |          11.1826 |
| Denoise    |          10.7183 |




# NIQE 

In [ ]:
def computeNIQE(task, DEBLOCKED_PATH):
    """ Compute NIQE for a specified task
    
    task : CycleGAN or U-GAT-IT
    DEBLOCKED_PATH : deblocked path for the task
    """
    deblocked_niqe = []
    deblocked_imgs = os.listdir(DEBLOCKED_PATH)

    for deblocked_img in deblocked_imgs:
        path = os.path.join(DEBLOCKED_PATH, deblocked_img)
        img = np.array(Image.open(path).convert("L")) # Convert to greyscale
        img = img[:,:,np.newaxis]
        niqe = measure.niqe(img)
        deblocked_niqe.append(niqe)

    print(f"{task} NIQE: ", np.mean(deblocked_niqe))

In [ ]:
# CycleGAN
computeNIQE("CycleGAN", CYCLEGAN_DEBLOCKED_PATH)

# U-GAT-IT TODO
computeNIQE("U-GAT-IT", UGATIT_DEBLOCKED_PATH)

CycleGAN NIQE:  34.651234
U-GAT-IT NIQE:  48.13686


# FID

In [ ]:
"""Implementation of FID https://github.com/toshas/torch-fidelity"""

!fidelity --silent --fid vangogh_originals cyclegan_vangogh
print("\n")
!fidelity --silent --fid vangogh_originals ugatit_vangogh

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
frechet_inception_distance: 250.1941


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
frechet_inception_distance: 305.9866


# KID

In [ ]:
#!git clone https://github.com/abdulfatir/gan-metrics-pytorch

fatal: destination path 'gan-metrics-pytorch' already exists and is not an empty directory.


In [ ]:
"""Implementation of KID with a slightly modified version of https://github.com/abdulfatir/gan-metrics-pytorch"""

!python gan-metrics-pytorch/kid_score.py --true vangogh_originals --fake cyclegan_vangogh
print("\n")
!python gan-metrics-pytorch/kid_score.py --true vangogh_originals --fake ugatit_vangogh

Namespace(batch_size=50, dims=2048, fake=['cyclegan_vangogh'], gpu='', model='inception', true='vangogh_originals')
100% 1/1 [00:12<00:00, 12.36s/it]
cyclegan_vangogh
100% 1/1 [00:12<00:00, 12.04s/it]
MMD: 100% 100/100 [00:52<00:00,  1.92it/s, mean=0.0789]
KID (cyclegan_vangogh): 0.079 (0.002)


Namespace(batch_size=50, dims=2048, fake=['ugatit_vangogh'], gpu='', model='inception', true='vangogh_originals')
100% 1/1 [00:12<00:00, 12.13s/it]
ugatit_vangogh
100% 1/1 [00:11<00:00, 11.98s/it]
MMD: 100% 100/100 [00:51<00:00,  1.94it/s, mean=0.134]
KID (ugatit_vangogh): 0.134 (0.003)
